In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
import base64
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
from AnimalShelter import AnimalShelter




###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
data = shelter.readAll({})
df = pd.DataFrame.from_records(data, columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.A(href="https://www.snhu.edu", children=html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()),
                                                          style={'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto', 'width': 'auto', 'height': 'auto'})),
    html.Center(html.B(html.H1('Jonathan Santiago\'s SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className='row',
             style={'flex-wrap': 'nowrap'},
             children=[dcc.Dropdown(id='dropdown-button', options=[
                 {"label": "All Rescue Types", "value": "Reset"},
                 {"label": "Water", "value": "Water"},
                 {"label": "Mountain", "value": "MWild"},
                 {"label": "Wilderness", "value": "MWild"},
                 {"label": "Disaster", "value": "DI"},
                 {"label": "Individual Tracking", "value": "DI"},
             ],
                                    value='Reset',
                                    clearable=False
                                    )
                       ]),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": True, "selectable": True} for i in df.columns
        ], 
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_deletable=False,
        row_selectable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10
    ),
    html.Br(),
    html.Hr(),
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

#Callback to filter data table from dropdown box
@app.callback(
    Output('datatable-id', 'data'),
    [Input('dropdown-button', 'value')])
def on_click(radio_value):
    global df
    global data
    if radio_value == "Water":
        data = shelter.readAll({"animal_type": "Dog",
                                "breed": {"$in": 
                                        ["Labrador Retriever Mix",
                                         "Labrador Retriever/Akita",
                                         "Labrador Retriever/American Bulldog",
                                         "Labrador Retriever/American Staffordshire Terrier",
                                         "Labrador Retriever/Anatol Shepherd",
                                         "Labrador Retriever/Australian Cattle Dog",
                                         "Labrador Retriever/Australian Kelpie",
                                         "Labrador Retriever/Australian Shepherd",
                                         "Labrador Retriever/Basset Hound",
                                         "Labrador Retriever/Beagle",
                                         "Labrador Retriever/Black Mouth Cur",
                                         "Labrador Retriever/Black/Tan Hound",
                                         "Labrador Retriever/Blue Lacy",
                                         "Labrador Retriever/Border Collie",
                                         "Labrador Retriever/Border Terrier",
                                         "Labrador Retriever/Boxer",
                                         "Labrador Retriever/Catahoula",
                                         "Labrador Retriever/Chesa Bay Retr",
                                         "Labrador Retriever/Chinese Sharpei",
                                         "Labrador Retriever/Chow Chow",
                                         "Labrador Retriever/Doberman Pinsch",
                                         "Labrador Retriever/English Springer Spaniel",
                                         "Labrador Retriever/Flat Coat Retriever",
                                         "Labrador Retriever/German Shepherd",
                                         "Labrador Retriever/Golden Retriever",
                                         "Labrador Retriever/Great Dane",
                                         "Labrador Retriever/Great Pyrenees",
                                         "Labrador Retriever/Harrier",
                                         "Labrador Retriever/Labrador Retriever",
                                         "Labrador Retriever/Mastiff",
                                         "Labrador Retriever/Miniature Poodle",
                                         "Labrador Retriever/Newfoundland",
                                         "Labrador Retriever/Pembroke Welsh Corgi",
                                         "Labrador Retriever/Pit Bull",
                                         "Labrador Retriever/Plott Hound",
                                         "Labrador Retriever/Pointer",
                                         "Labrador Retriever/Queensland Heeler",
                                         "Labrador Retriever/Redbone Hound",
                                         "Labrador Retriever/Rottweiler",
                                         "Labrador Retriever/Siberian Husky",
                                         "Labrador Retriever/Smooth Fox Terrier",
                                         "Labrador Retriever/Staffordshire",
                                         "Labrador Retriever/Vizsla",
                                         "Labrador Retriever/Whippet",
                                         "Chesa Bay Retr Mix", 
                                         "Newfoundland Mix",
                                         "Newfoundland/Australian Cattle Dog",
                                         "Newfoundland/Great Pyrenees",
                                         "Newfoundland/Labrador Retriever"]},
                                "sex_upon_outcome": "Intact Female",
                                "age_upon_outcome_in_weeks": {"$gt": 25, "$lt": 157}})
    elif radio_value == "MWild":
        data = shelter.readAll({"animal_type": "Dog",
                                "breed": {"$in":
                                          ["German Shepherd",
                                           "German Shepherd Mix",
                                           "German Shepherd/Alaskan Husky",
                                           "German Shepherd/Alaskan Malamute",
                                           "German Shepherd/Anatol Shepherd",
                                           "German Shepherd/Australian Cattle Dog",
                                           "German Shepherd/Australian Kelpie",
                                           "German Shepherd/Black/Tan Hound",
                                           "German Shepherd/Border Collie",
                                           "German Shepherd/Boxer",
                                           "German Shepherd/Cardigan Welsh Corgi",
                                           "German Shepherd/Chow Chow",
                                           "German Shepherd/Collie Rough",
                                           "German Shepherd/Doberman Pinsch",
                                           "German Shepherd/Labrador Retriever",
                                           "German Shepherd/Pit Bull",
                                           "German Shepherd/Rottweiler",
                                           "German Shepherd/Siberian Husky",
                                           "Alaskan Malamute",
                                           "Alaskan Malamute Mix",
                                           "Alaskan Malamute/Border Collie",
                                           "Alaskan Malamute/Labrador Retriever",
                                           "Old English Sheepdog",
                                           "Siberian Husky",
                                           "Siberian Husky Mix",
                                           "Siberian Husky/Anatol Shepherd",
                                           "Siberian Husky/Border Collie",
                                           "Siberian Husky/German Shepherd",
                                           "Siberian Husky/Labrador Retriever",
                                           "Rottweiler",
                                           "Rottweiler Mix",
                                           "Rottweiler/German Shepherd",
                                           "Rottweiler/Labrador Retriever",
                                           "Rottweiler/Rhod Ridgeback",
                                           "Rottweiler/Siberian Husky",
                                          ]},
                                "sex_upon_outcome": "Intact Male",
                                "age_upon_outcome_in_weeks": {"$gt": 25, "$lt": 157}})
    elif radio_value == "DI":
        data = shelter.readAll({"animal_type": "Dog",
                                "breed": {"$in":
                                          ["Doberman Pinsch",
                                           "Doberman Pinsch Mix",
                                           "Doberman Pinsch/Australian Cattle Dog",
                                           "Doberman Pinsch/Catahoula",
                                           "Doberman Pinsch/Vizsla",
                                           "German Shepherd",
                                           "German Shepherd Mix",
                                           "German Shepherd/Alaskan Husky",
                                           "German Shepherd/Alaskan Malamute",
                                           "German Shepherd/Anatol Shepherd",
                                           "German Shepherd/Australian Cattle Dog",
                                           "German Shepherd/Australian Kelpie",
                                           "German Shepherd/Black/Tan Hound",
                                           "German Shepherd/Border Collie",
                                           "German Shepherd/Boxer",
                                           "German Shepherd/Cardigan Welsh Corgi",
                                           "German Shepherd/Chow Chow",
                                           "German Shepherd/Collie Rough",
                                           "German Shepherd/Doberman Pinsch",
                                           "German Shepherd/Labrador Retriever",
                                           "German Shepherd/Pit Bull",
                                           "German Shepherd/Rottweiler",
                                           "German Shepherd/Siberian Husky"
                                           "Golden Retriever",
                                           "Golden Retriever Mix",
                                           "Golden Retriever/Border Collie",
                                           "Golden Retriever/Chow Chow",
                                           "Golden Retriever/Dachshund",
                                           "Golden Retriever/Great Pyrenees",
                                           "Golden Retriever/Labrador Retriever",
                                           "Golden Retriever/Pembroke Welsh Corgi",
                                           "Golden Retriever/Whippet",
                                           "Bloodhound",
                                           "Bloodhound Mix",
                                           "Rottweiler Mix",
                                           "Rottweiler/German Shepherd",
                                           "Rottweiler/Labrador Retriever",
                                           "Rottweiler/Rhod Ridgeback",
                                           "Rottweiler/Siberian Husky"]},
                                "sex_upon_outcome": "Intact Male",
                                "age_upon_outcome_in_weeks": {"$gt": 19, "$lt": 301}})
    elif radio_value == "Reset":
        data = shelter.readAll({"animal_type": "Dog"})

    df = pd.DataFrame.from_records(data,
                                   columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                            'date_of_birth', 'datetime', 'monthyear', 'name',
                                            'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                            'age_upon_outcome_in_weeks'])
    return df.to_dict('records')




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# pie chart graph of animal types
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(view_data):
    dff = pd.DataFrame.from_dict(view_data)
    breeds = []
    for i in range(0, dff.__len__() - 1):
        breeds.append(str(dff.iloc[i, 4]))
    return [
        dcc.Graph(
            figure={
                'data': [
                    {
                        'labels': breeds,
                        'type': 'pie'
                    }
                ],
                'layout': {
                    'title': 'Breeds',
                }
            }
        )
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(view_data):
    if view_data is None:
        return []
    else:
        dff = pd.DataFrame.from_dict(view_data)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[dff.iloc[0, 12], dff.iloc[0, 13]], zoom=10,
                   children=[
                       dl.TileLayer(id="base-layer-id"),
                       # Marker with tool tip and popup
                       dl.Marker(position=[dff.iloc[0, 12], dff.iloc[0, 13]], children=[
                           dl.Tooltip(dff.iloc[0, 4]),
                           dl.Popup([
                               html.H1(dff.iloc[0, 9]),
                               html.P(dff.iloc[0, 8])
                           ])
                       ])
                   ])
        ]

app